In [ ]:
import numpy as np
import time

## Generate the Data

We first generate a random dataset with number of features (m = 10) and number of instances (n = 100)
We also generate a random label vector y \in {-1,1}

In [ ]:
n = 100
m = 10

X = np.random.rand(n,m)
y = np.random.rand(n)
y = np.random.rand(n)
ybin = [(int(yi >= 0.5) - int(yi < 0.5)) for yi in y]
y = np.array(ybin)
w = np.random.rand(m, 1)
print(y)

## A Simple naive Implementation of the Least Squares

Below is a simple naive implementation of Least Square Loss. We directly plug in the formula with a simple for loop!

In [ ]:
def HingeLossNaive(w, X, y, lam):
    # Computes the cost function for all the training samples
    f = 0
    g = 0
    for i in range(len(X)):
        featureweightProd = np.dot(X[i],w)
        f = f + np.max([0, 1 - y[i]*featureweightProd])
        g = g - y[i]*X[i]*np.double(1 > y[i]*featureweightProd) 
    f = f + 0.5*lam*np.sum(w*w)
    g = g + lam*w.reshape(1,-1)
    return [f, g]     

In [ ]:
start = time.time()
[f,g] = HingeLossNaive(w,X,y,1)
end = time.time()
print("Time Taken = " + str(end - start))
print("Function value = " + str(f))
print("Printing Gradient:")
print(g)

## For Loop in Python == Slow Code

Great, we have a working code now. But while this code might be correct, is this going to be fast? We have a For loop in python which is clearly an issue!

First let us see how slow the code is! Let us increase n to 10000000 and m to 1000, which are somewhat more realistic (though still far from real world).

In [ ]:
n = 1000000
m = 100

X = np.random.rand(n,m)
y = np.random.rand(n)
y = np.random.rand(n)
ybin = [(int(yi >= 0.5) - int(yi < 0.5)) for yi in y]
y = np.array(ybin)
w = np.random.rand(m, 1)

start = time.time()
[f,g] = HingeLossNaive(w,X,y,1)
end = time.time()
print("Time Taken = " + str(end - start))
print("Function value = " + str(f))
print(g)

## Speeding up the code!

With n = 10000000, it takes around 4.5 minutes to run a single function evaluation!

Lets now vectorize the code below.

In [ ]:
Xw = np.matmul(X,w)
yT = y.reshape(-1,1)
yXw = np.multiply(yT,Xw)
np.shape(yXw)
#f = np.sum(np.max(0, 1 - yXw.T)) + 0.5*np.sum(w*w)
#print(f)
#ymul = -1*yT*np.double(1 > yXw) 
#print(np.shape(ymul.reshape(1,-1)))
#print(np.shape(X))
#g = np.matmul(ymul.reshape(1,-1),X).reshape(-1,1)  + 1*w.reshape(-1,1)
#print(g)

In [ ]:
def HingeLoss(w, X, y, lam):
    # Computes the cost function for all the training samples
    Xw = np.matmul(X,w)
    yT = y.reshape(-1,1)
    yXw = np.multiply(yT,Xw)
    f = np.sum(np.maximum(0, 1 - yXw.T)) + 0.5*np.sum(w*w)
    ymul = -1*yT*np.double(1 > yXw)    
    g = np.matmul(ymul.reshape(1,-1),X).reshape(-1,1)  + 1*w.reshape(-1,1)
    return [f, g]

In [ ]:
start = time.time()
[f,g] = HingeLoss(w,X,y,1)
end = time.time()
print("Time Taken = " + str(end - start))
print("Function value = " + str(f))
print(g)

## Checking gradient implementations!

So far so good! But how do we verify if our gradient implementation is correct?
We can test out our loss function analytically, but what if we make a mistake in computing the gradient? We can numerically compute the gradient to ensure it is correct.

In [ ]:
def LeastSquaresFun(w, X, y, lam):
    # Computes the cost function for all the training samples
    m = X.shape[0]
    Xw = np.matmul(X,w)
    Xwy = (Xw - y).reshape(-1,1)
    f = np.dot(Xwy.T,Xwy) + 0.5*lam*np.sum(w*w)
    return f

def numericalGrad(funObj, w,epsilon):
    m = len(w)
    grad = np.zeros(m)
    for i in range(m):
        wp = np.copy(w)
        wn = np.copy(w)
        wp[i] = w[i] + epsilon
        wn[i] = w[i] - epsilon
        grad[i] = (funObj(wp) - funObj(wn))/(2*epsilon)
    return grad

In [ ]:
n = 100
m = 10

X = np.random.rand(n,m)
wgen = np.random.rand(m)
y = np.dot(X,wgen) + np.random.normal(0, 0.1, n)
w = np.random.rand(m)

funObj = lambda w: LeastSquaresFun(w,X,y,1)
[f,g] = LeastSquares(w,X,y,1)
gn = numericalGrad(funObj, w, 1e-10)
fn = funObj(w)
print(f)
print(fn)
print(gn)
print(g)